## Preparations

In [30]:
# !auto-py-to-exe

In [31]:
import sys

# Определяем, использовать ли print вместо display
if hasattr(sys, 'frozen'):  # Если код выполняется в собранном файле
    def display(*args, **kwargs):
        print(*args, **kwargs)

In [32]:
import warnings
warnings.filterwarnings("ignore")

In [33]:
print("d2loadout update started")

d2loadout update started


In [34]:
import json
import os
import re
import pandas as pd
import numpy as np
import requests
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import ast
import html_to_json

In [35]:
chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument("user-agent=" + UserAgent().random)
chrome_options.add_argument("--disable-blink-features")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("disable-infobars")
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("detach", True)

chrome_options.add_argument('headless')
chrome_options.add_argument('window-size=1920x1080')
chrome_options.add_argument("disable-gpu")

chrome_options.page_load_strategy = 'eager'


os.environ["PATH"] += r"C:\Program Files (x86)\Chromedriver.exe"
driver = webdriver.Chrome(options=chrome_options)

## Meta

In [36]:
link_meta = "https://dota2protracker.com/meta"
driver.implicitly_wait(3)
driver.get(link_meta)
driver.implicitly_wait(10)

In [37]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
import time

def get_d2pt_page_table(driver):
    # Даем время на загрузку контента
    time.sleep(0.2)

    # Получаем HTML страницы
    page_source = driver.page_source

    # Парсим HTML с помощью BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Ищем строки таблицы по их CSS-классу
    table_rows = soup.find_all('div', class_='grid', style=True)

    # Список для хранения данных
    data = []
    headers = []

    for index, row in enumerate(table_rows):
        # Проверяем, чтобы строка не была заголовком
        cols = row.find_all('div', recursive=False)

        if index == 0:
            # Считаем первую строку заголовком
            for col in cols:
                header_text = col.get_text(strip=True)
                headers.append(header_text if header_text else None)
            continue

        row_data = []
        for col in cols:
            # Проверяем наличие изображений для извлечения роли
            img = col.find('img', alt=True)
            if img:
                row_data.append(img['alt'])
                continue

            # Обрабатываем содержимое внутри span
            spans = col.find_all('span')
            if spans:
                span_text = ' '.join([span.get_text(strip=True) for span in spans])
                row_data.append(span_text if span_text else None)
                continue

            # Извлекаем текст и убираем пустые значения
            text = col.get_text(strip=True)
            row_data.append(text if text else None)

        if row_data:  # Только если данные не пусты
            data.append(row_data)

    # Преобразуем список в DataFrame
    df_heroes_table = pd.DataFrame(data, columns=headers)

    # Убираем лишние строки или обрабатываем некорректные данные
    df_heroes_table = df_heroes_table.dropna(how='all')

    # Очистка данных в процентах и диапазонах
    def clean_data(value):
        if isinstance(value, str):
            if '%' in value:
                try:
                    return float(value.replace('%', ''))
                except ValueError:
                    return value  # Вернуть исходное значение, если не удалось преобразовать
            elif '(' in value and ')' in value:
                try:
                    main_value, range_values = value.split('(')
                    main_value = main_value.strip()
                    range_values = range_values.replace(')', '').split('-')
                    return {
                        'main': int(main_value),
                        'range_min': int(range_values[0]),
                        'range_max': int(range_values[1])
                    }
                except ValueError:
                    return value  # Вернуть исходное значение, если формат не соответствует
        return value

    # Применяем очистку данных ко всем ячейкам
    df_heroes_table = df_heroes_table.map(clean_data)

    # Обработка типов данных для чисел
    df_heroes_table = df_heroes_table.apply(lambda x: x.convert_dtypes() if x.name != 2 else x)

    return df_heroes_table

# Пример использования:
# driver = ...  # Selenium WebDriver
# df = get_d2pt_page_table(driver)
# print(df)


In [38]:
print("checking meta...")
print("fetching heroes info for pos 1")
driver.find_element(By.XPATH, "//div[contains(text(), 'Carry')]").click()
df_1 = get_d2pt_page_table(driver)

print("fetching heroes info for pos 2")
driver.find_element(By.XPATH, "//div[contains(text(), 'Mid')]").click()
df_2 = get_d2pt_page_table(driver)

print("fetching heroes info for pos 3")
driver.find_element(By.XPATH, "//div[contains(text(), 'Off')]").click()
df_3 = get_d2pt_page_table(driver)

print("fetching heroes info for pos 4")
driver.find_element(By.XPATH, "//div[contains(text(), 'Pos 4')]").click()
df_4 = get_d2pt_page_table(driver)

print("fetching heroes info for pos 5")
driver.find_element(By.XPATH, "//div[contains(text(), 'Pos 5')]").click()
df_5 = get_d2pt_page_table(driver)

df_full = pd.concat([df_1, df_2, df_3, df_4, df_5], axis=0)
df_full

checking meta...
fetching heroes info for pos 1
fetching heroes info for pos 2
fetching heroes info for pos 3
fetching heroes info for pos 4
fetching heroes info for pos 5


,Rank,Role,Hero,D2PT Rating,Matches,Win Rate,WR 9500+ MMR,Contest Rate,Radiant,Dire,Expert,1st Phase,2nd Phase,Lastpick,Networth@10
0,#1,pos 1,Bloodseeker,3449,3551,54.1,53.5% 1870 -0.6%,36.7,56.8,51.3,59.5% (788),49.5,54.4,54.8,"{'main': 4305, 'range_min': 3625, 'range_max':..."
1,#2,pos 1,Alchemist,3389,4959,52.9,54.4% 2624 +1.5%,58.6,53.4,52.4,57.7% (889),50.5,53.7,53.0,"{'main': 5571, 'range_min': 4649, 'range_max':..."
2,#3,pos 1,Spectre,3309,4542,52.5,54.2% 2035 +1.7%,63.7,53.7,51.2,57.1% (609),50.3,52.8,53.8,"{'main': 3915, 'range_min': 3261, 'range_max':..."
3,#4,pos 1,Marci,3098,278,57.9,56.6% 113 -1.3%,19.5,54.1,60.9,74.1% (27),65.0,50.9,51.9,"{'main': 4810, 'range_min': 3799, 'range_max':..."
4,#5,pos 1,Medusa,3092,885,52.7,46.5% 344 -6.1%,14.0,52.8,52.5,63.8% (94),66.7,54.4,50.6,"{'main': 4495, 'range_min': 3817, 'range_max':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,#122,pos 5,Underlord,0,3,0.0,0.0% 1 +0.0%,6.1,0.0,0.0,0.0% (0),0.0,0.0,0.0,"{'main': 1818, 'range_min': 1599, 'range_max':..."
122,#123,pos 5,Pangolier,0,7,14.3,16.7% 6 +2.4%,13.0,0.0,50.0,0.0% (0),33.3,0.0,0.0,"{'main': 1604, 'range_min': 1604, 'range_max':..."
123,#124,pos 5,Mars,0,15,53.3,60.0% 5 +6.7%,17.4,50.0,57.1,0.0% (0),62.5,60.0,0.0,"{'main': 2450, 'range_min': 1354, 'range_max':..."
124,#125,pos 5,Void Spirit,0,10,10.0,0.0% 2 -10.0%,7.7,25.0,0.0,0.0% (0),16.7,0.0,0.0,"{'main': 2564, 'range_min': 1719, 'range_max':..."


## Facets

In [39]:
link = "https://dota2protracker.com/facets"

In [40]:
driver.implicitly_wait(2)
driver.get(link)

In [41]:
hero_req = requests.get("https://dota2protracker.com/hero/Tiny")

In [42]:
import re

# Пример HTML-кода (замени на свой)
html_text = hero_req.text
# Регулярное выражение для извлечения объекта facetData
pattern = r'facetData:\s*({.*?})\s*\}\s*,'
constdata = re.findall(pattern, html_text, re.DOTALL)[0]

constdata


'{"1":{id:"1",localized_name:"Anti-Mage",icon:"",facets:[{name:"Magebane\'s Mirror",description:"Counterspell and Counterspell Ally reflect spells back at their caster instead of just blocking.",background:"linear-gradient(to right, rgb(156, 112, 164), rgb(40, 39, 82))",deprecated:false,icon:"https://dota2protracker.com/static/facet_icons/ricochet.png"},{name:"Mana Thirst",description:"Anti-Mage gains damage when his enemies are at low mana.",background:"linear-gradient(to right, rgb(148, 181, 186), rgb(56, 91, 89))",deprecated:false,icon:"https://dota2protracker.com/static/facet_icons/mana.png"}]},"2":{id:"2",localized_name:"Axe",icon:"",facets:[{name:"One Man Army",description:"Axe gains Strength based on his armor as long as he\'s away from his allies.",background:"linear-gradient(to right, rgb(159, 60, 60), rgb(74, 32, 64))",deprecated:false,icon:"https://dota2protracker.com/static/facet_icons/strength.png"},{name:"Call Out",description:"Berserker\'s Call gives attacking enemies mo

In [43]:
import js2py
import json
import pandas as pd

# Пример JavaScript-кода
js_code = f"""
var data = {constdata};

// Возвращаем JSON-строку facetData
JSON.stringify(data);
"""

# Исполняем JavaScript-код
result = js2py.eval_js(js_code)

if result:
    # Преобразуем JSON-строку в Python-словарь
    facet_data = json.loads(result)
    
    # Инициализируем пустой DataFrame
    all_facets_df = pd.DataFrame()

    # Итерация по всем ключам в facetData
    for key, value in facet_data.items():
        facets = value.get("facets", [])
        # Преобразуем каждый набор facets в DataFrame
        facets_data = []
        for facet in facets:
            if not facet.get("deprecated", False):  # Пропускаем deprecated
                facets_data.append(facet)
        
        facets_df = pd.DataFrame(facets_data)
        
        # Добавляем информацию о ключе, id и localized_name в DataFrame
        facets_df["key"] = key  # Ключ из facetData
        facets_df["id"] = value.get("id")
        facets_df["localized_name"] = value.get("localized_name")
        
        # Объединяем текущий DataFrame с общим
        all_facets_df = pd.concat([all_facets_df, facets_df], ignore_index=True)
    
    # Перенумерация facet_number
    all_facets_df["facet_number"] = (
        all_facets_df.groupby(["key", "id"]).cumcount() + 1
    )
    
    # Выводим общий DataFrame
    display(all_facets_df)
else:
    print("Данные не найдены или невалидны.")


,background,deprecated,description,icon,name,key,id,localized_name,num_matches,pickRate,winRate,facet_number
0,"linear-gradient(to right, rgb(156, 112, 164), ...",False,Counterspell and Counterspell Ally reflect spe...,https://dota2protracker.com/static/facet_icons...,Magebane's Mirror,1,1,Anti-Mage,NaN,NaN,NaN,1
1,"linear-gradient(to right, rgb(148, 181, 186), ...",False,Anti-Mage gains damage when his enemies are at...,https://dota2protracker.com/static/facet_icons...,Mana Thirst,1,1,Anti-Mage,NaN,NaN,NaN,2
2,"linear-gradient(to right, rgb(162, 178, 62), r...",False,Morphling's Primary Attribute is now <font col...,https://dota2protracker.com/static/facet_icons...,Ebb,10,10,Morphling,NaN,NaN,NaN,1
3,"linear-gradient(to right, rgb(159, 60, 60), rg...",False,Morphling's Primary Attribute is now <font col...,https://dota2protracker.com/static/facet_icons...,Flow,10,10,Morphling,NaN,NaN,NaN,2
4,"linear-gradient(to right, rgb(148, 181, 186), ...",False,Activate to temporarily apply a negative debuf...,https://dota2protracker.com/static/facet_icons...,Tag Team,100,100,Tusk,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
259,"linear-gradient(to right, rgb(162, 178, 62), r...",False,Timber Chain splinters the tree into all direc...,https://dota2protracker.com/static/facet_icons...,Shredder,98,98,Timbersaw,NaN,NaN,NaN,1
260,"linear-gradient(to right, rgb(159, 60, 60), rg...",False,Launches a second independent Chakram that fol...,https://dota2protracker.com/static/facet_icons...,Twisted Chakram,98,98,Timbersaw,NaN,NaN,NaN,2
261,"linear-gradient(to right, rgb(202, 193, 148), ...",False,Warpath grants scaling Attack Speed instead of...,https://dota2protracker.com/static/facet_icons...,Berserk,99,99,Bristleback,NaN,NaN,NaN,1
262,"linear-gradient(to right, rgb(162, 178, 62), r...",False,Bristleback now fires Viscous Nasal Goo instea...,https://dota2protracker.com/static/facet_icons...,Snot Rocket,99,99,Bristleback,NaN,NaN,NaN,2


In [44]:
def get_d2pt_page_table_facets(driver):
    time.sleep(0.2)
    # finding table rows in site and converting to format, readable by pandas
    category_name_elements = driver.find_elements(
        By.CSS_SELECTOR, ".flex.bg-d2pt-gray-3.gap-1"
    )
    category_name_elements += driver.find_elements(
        By.CSS_SELECTOR, ".flex.bg-d2pt-gray-4.gap-1"
    )
    hero_rows = [item.text.split("\n") for item in category_name_elements]
    hero_columns = [item for item in driver.find_element(
        By.CSS_SELECTOR, ".flex.gap-1.font-medium.text-sm.mb-1"
    ).text.split("\n") if item != "Trend"]
        
    # Dataframe dtype converts
    df_heroes_table = pd.DataFrame(data=hero_rows, columns=hero_columns)
    df_heroes_table = df_heroes_table.convert_dtypes()
    df_heroes_table = df_heroes_table.round(1)
    return df_heroes_table

In [45]:
print("checking facets...")
print("fetching facets info for pos 1")
driver.find_element(By.XPATH, "//div[contains(text(), 'Carry')]").click()
df_1 = get_d2pt_page_table_facets(driver)
df_1["Role"] = "pos 1"

print("fetching facets info for pos 2")
driver.find_element(By.XPATH, "//div[contains(text(), 'Mid')]").click()
df_2 = get_d2pt_page_table_facets(driver)
df_2["Role"] = "pos 2"

print("fetching facets info for pos 3")
driver.find_element(By.XPATH, "//div[contains(text(), 'Off')]").click()
df_3 = get_d2pt_page_table_facets(driver)
df_3["Role"] = "pos 3"

print("fetching facets info for pos 4")
driver.find_element(By.XPATH, "//div[contains(text(), 'Pos 4')]").click()
df_4 = get_d2pt_page_table_facets(driver)
df_4["Role"] = "pos 4"

print("fetching facets info for pos 5")
driver.find_element(By.XPATH, "//div[contains(text(), 'Pos 5')]").click()
df_5 = get_d2pt_page_table_facets(driver)
df_5["Role"] = "pos 5"

df_full_facets = pd.concat([df_1,df_2,df_3,df_4,df_5], axis=0)

df_full_facets = df_full_facets.rename({
    "Hero": "hero",
    "Facet": "facet",
    "Matches": "Matches",
    "Win Rate": "Win Rate"
}, axis=1)[["hero", "facet", "Matches", "Win Rate", "Role"]]

df_full_facets["Win Rate"] = df_full_facets["Win Rate"].apply(lambda x: x.replace("%", ""))
df_full_facets = df_full_facets.convert_dtypes()

df_full_facets

checking facets...
fetching facets info for pos 1
fetching facets info for pos 2
fetching facets info for pos 3
fetching facets info for pos 4
fetching facets info for pos 5


,hero,facet,Matches,Win Rate,Role
0,Tidehunter,KRAKEN SWELL,22,54.5,pos 1
1,Warlock,BLACK GRIMOIRE,1,100.0,pos 1
2,Bounty Hunter,THROUGH AND THROUGH,7,0.0,pos 1
3,Spirit Breaker,BULL RUSH,3,0.0,pos 1
4,Rubick,FRUGAL FILCH,2,0.0,pos 1
...,...,...,...,...,...
259,Clinkz,SUPPRESSIVE FIRE,6,66.7,pos 5
260,Storm Spirit,STATIC SLIDE,8,37.5,pos 5
261,Slardar,LEG DAY,8,62.5,pos 5
262,Faceless Void,TIME ZONE,10,40.0,pos 5


## Results

In [46]:
print("creating in-game loadouts")

creating in-game loadouts


In [47]:
import pandas as pd


class HeroConfigProcessor:
    def __init__(self, df, name, data_type="facet"):
        """
        Класс для обработки и создания конфигураций на основе DataFrame.

        Args:
            df (pd.DataFrame): DataFrame с данными героев.
            name (str): Имя конфигурации.
            data_type (str): Тип данных ('facet' или 'regular').
        """
        self.df = df
        self.name = name
        self.data_type = data_type

    def get_hero_ids(
        self,
        position,
        facet_number=None,
        facet_id=None,
        wr_threshold=50,
        matches_threshold=50,
        expert_matches_threshold=None,
        expert_wr_threshold=None,
        mmr_9500_matches_threshold=None,
        mmr_9500_wr_threshold=None,
        rating_threshold=None,
        sort_by="Win Rate",
        ascending=False,
    ):
        """
        Получает список идентификаторов героев для указанной позиции.

        Args:
            position (int): Позиция героя (1 для pos 1, 2 для pos 2 и т.д.).
            facet_number (int, optional): Номер фасета.
            facet_id (int, optional): Идентификатор фасета (название фасета).
            wr_threshold (float, optional): Минимальный win rate.
            matches_threshold (int, optional): Минимальное количество матчей.
            expert_matches_threshold (int, optional): Минимальное количество экспертных матчей.
            expert_wr_threshold (float, optional): Минимальный экспертный win rate.
            mmr_9500_matches_threshold (int, optional): Минимальное количество матчей для 9500 MMR.
            mmr_9500_wr_threshold (float, optional): Минимальный win rate для 9500 MMR.
            rating_threshold (float, optional): Минимальный D2PT рейтинг.
            sort_by (str, optional): Поле для сортировки.
            ascending (bool, optional): Порядок сортировки.

        Returns:
            list: Список идентификаторов героев.
        """
        position_str = f"pos {position}"

        # Фильтруем данные по базовым критериям
        filtered_df = self.df[
            (self.df["Role"] == position_str)
            & (self.df["Matches"] > matches_threshold)
            & (self.df["Win Rate"] > wr_threshold)
        ]

        # Дополнительные фильтры
        if expert_matches_threshold is not None and "Expert Matches" in self.df.columns:
            filtered_df = filtered_df[
                filtered_df["Expert Matches"] > expert_matches_threshold
            ]
        if expert_wr_threshold is not None and "Expert Win Rate" in self.df.columns:
            filtered_df = filtered_df[
                filtered_df["Expert Win Rate"] > expert_wr_threshold
            ]
        if mmr_9500_matches_threshold is not None and "9500 Matches" in self.df.columns:
            filtered_df = filtered_df[
                filtered_df["9500 Matches"] > mmr_9500_matches_threshold
            ]
        if mmr_9500_wr_threshold is not None and "9500 Win Rate" in self.df.columns:
            filtered_df = filtered_df[
                filtered_df["9500 Win Rate"] > mmr_9500_wr_threshold
            ]
        if rating_threshold is not None and "D2PT Rating" in self.df.columns:
            filtered_df = filtered_df[filtered_df["D2PT Rating"] > rating_threshold]

        # Фильтрация для фасетов
        if self.data_type == "facet":
            if facet_number is not None:
                if facet_number == "3+":
                    filtered_df = filtered_df[filtered_df["facet_number"] > 2]
                else:
                    filtered_df = filtered_df[
                        filtered_df["facet_number"] == facet_number
                    ]
            if facet_id is not None:
                filtered_df = filtered_df[filtered_df["facet"] == facet_id]

        # Сортировка данных
        if sort_by in filtered_df.columns:
            filtered_df = filtered_df.sort_values(by=sort_by, ascending=ascending)

        return filtered_df["hero_id"].tolist()

    def build_config(
        self,
        wr_threshold=50,
        matches_threshold=50,
        expert_matches_threshold=None,
        expert_wr_threshold=None,
        mmr_9500_matches_threshold=None,
        mmr_9500_wr_threshold=None,
        rating_threshold=None,
        sort_by="Win Rate",
        ascending=False,
    ):
        """
        Создает конфигурацию на основе обработанных данных.

        Args:
            wr_threshold (float, optional): Минимальный win rate.
            matches_threshold (int, optional): Минимальное количество матчей.
            expert_matches_threshold (int, optional): Минимальное количество экспертных матчей.
            expert_wr_threshold (float, optional): Минимальный экспертный win rate.
            mmr_9500_matches_threshold (int, optional): Минимальное количество матчей для 9500 MMR.
            mmr_9500_wr_threshold (float, optional): Минимальный win rate для 9500 MMR.
            rating_threshold (float, optional): Минимальный D2PT рейтинг.
            sort_by (str, optional): Поле для сортировки.
            ascending (bool, optional): Порядок сортировки.

        Returns:
            dict: Конфигурация категорий.
        """
        if self.data_type == "facet":
            return self._build_facet_config(
                wr_threshold,
                matches_threshold,
                expert_matches_threshold,
                expert_wr_threshold,
                mmr_9500_matches_threshold,
                mmr_9500_wr_threshold,
                rating_threshold,
                sort_by,
                ascending,
            )
        elif self.data_type == "regular":
            return self._build_regular_config(
                wr_threshold,
                matches_threshold,
                expert_matches_threshold,
                expert_wr_threshold,
                mmr_9500_matches_threshold,
                mmr_9500_wr_threshold,
                rating_threshold,
                sort_by,
                ascending,
            )
        else:
            raise ValueError(f"Unsupported data type: {self.data_type}")

    def _build_facet_config(
        self,
        wr_threshold,
        matches_threshold,
        expert_matches_threshold,
        expert_wr_threshold,
        mmr_9500_matches_threshold,
        mmr_9500_wr_threshold,
        rating_threshold,
        sort_by,
        ascending,
    ):
        margin = 20
        max_height = 570
        max_width = 1180 - 100
        width = max_width / 5 - margin
        height = max_height / 2 - margin

        height_3 = (max_height / 5) - 20
        width_3 = 70

        categories = []
        for position in range(1, 6):
            hero_ids_facet_1 = self.get_hero_ids(
                position,
                facet_number=1,
                wr_threshold=wr_threshold,
                matches_threshold=matches_threshold,
                expert_matches_threshold=expert_matches_threshold,
                expert_wr_threshold=expert_wr_threshold,
                mmr_9500_matches_threshold=mmr_9500_matches_threshold,
                mmr_9500_wr_threshold=mmr_9500_wr_threshold,
                rating_threshold=rating_threshold,
                sort_by=sort_by,
                ascending=ascending,
            )
            hero_ids_facet_2 = self.get_hero_ids(
                position,
                facet_number=2,
                wr_threshold=wr_threshold,
                matches_threshold=matches_threshold,
                expert_matches_threshold=expert_matches_threshold,
                expert_wr_threshold=expert_wr_threshold,
                mmr_9500_matches_threshold=mmr_9500_matches_threshold,
                mmr_9500_wr_threshold=mmr_9500_wr_threshold,
                rating_threshold=rating_threshold,
                sort_by=sort_by,
                ascending=ascending,
            )
            hero_ids_facet_3_plus = self.get_hero_ids(
                position,
                facet_number="3+",
                wr_threshold=wr_threshold,
                matches_threshold=matches_threshold,
                expert_matches_threshold=expert_matches_threshold,
                expert_wr_threshold=expert_wr_threshold,
                mmr_9500_matches_threshold=mmr_9500_matches_threshold,
                mmr_9500_wr_threshold=mmr_9500_wr_threshold,
                rating_threshold=rating_threshold,
                sort_by=sort_by,
                ascending=ascending,
            )

            if hero_ids_facet_1:
                categories.append(
                    {
                        "category_name": f"Pos {position} F 1",
                        "x_position": (width + margin) * (position - 1),
                        "y_position": 0,
                        "width": width,
                        "height": height,
                        "hero_ids": hero_ids_facet_1,
                    }
                )
            if hero_ids_facet_2:
                categories.append(
                    {
                        "category_name": f"Pos {position} F 2",
                        "x_position": (width + margin) * (position - 1),
                        "y_position": height + margin,
                        "width": width,
                        "height": height,
                        "hero_ids": hero_ids_facet_2,
                    }
                )
            if hero_ids_facet_3_plus:
                categories.append(
                    {
                        "category_name": f"Pos {position} F 3+",
                        "x_position": max_width + margin,
                        "y_position": (height_3 + margin) * (position - 1),
                        "width": width_3,
                        "height": height_3,
                        "hero_ids": hero_ids_facet_3_plus,
                    }
                )

        return {
            "config_name": self.name,
            "categories": categories,
        }

    def _build_regular_config(
        self,
        wr_threshold,
        matches_threshold,
        expert_matches_threshold,
        expert_wr_threshold,
        mmr_9500_matches_threshold,
        mmr_9500_wr_threshold,
        rating_threshold,
        sort_by,
        ascending,
    ):
        margin = 20
        width = 585 - margin
        height = 189 - margin

        included_hero_ids = set()
        categories = []

        for position in range(1, 6):
            hero_ids_regular = self.get_hero_ids(
                position,
                wr_threshold=wr_threshold,
                matches_threshold=matches_threshold,
                expert_matches_threshold=expert_matches_threshold,
                expert_wr_threshold=expert_wr_threshold,
                mmr_9500_matches_threshold=mmr_9500_matches_threshold,
                mmr_9500_wr_threshold=mmr_9500_wr_threshold,
                rating_threshold=rating_threshold,
                sort_by=sort_by,
                ascending=ascending,
            )

            included_hero_ids.update(hero_ids_regular)

            if position <= 3:
                categories.append(
                    {
                        "category_name": f"Regular Pos {position}",
                        "x_position": 0.0,
                        "y_position": (height + margin) * (position - 1),
                        "width": width,
                        "height": height,
                        "hero_ids": hero_ids_regular,
                    }
                )
            else:
                categories.append(
                    {
                        "category_name": f"Regular Pos {position}",
                        "x_position": width + margin,
                        "y_position": (height + margin) * (position - 4),
                        "width": width,
                        "height": height,
                        "hero_ids": hero_ids_regular,
                    }
                )

        all_hero_ids = set(self.df["hero_id"].unique())
        not_included_hero_ids = list(all_hero_ids - included_hero_ids)

        if not_included_hero_ids:
            categories.append(
                {
                    "category_name": "Not-Included",
                    "x_position": width + margin,
                    "y_position": (height + margin) * 2,
                    "width": width,
                    "height": height,
                    "hero_ids": not_included_hero_ids,
                }
            )

        return {
            "config_name": self.name,
            "categories": categories,
        }

In [48]:
# Оставляем только уникальные значения в правом DataFrame
all_facets_df_unique = all_facets_df.drop_duplicates(subset=["localized_name"], keep="first")

# Выполняем объединение
df_final = pd.merge(
    df_full,
    all_facets_df_unique[["id", "localized_name"]],  # Оставляем только нужные столбцы
    left_on=["Hero"],  # Соответствие по названию фасета и герою
    right_on=["localized_name"],  # Название фасета и героя
    how="left"  # Оставляем все строки из df_full
)

# Переименовываем столбец "id" в "hero_id"
df_final.rename(columns={"id": "hero_id"}, inplace=True)

# Перемещаем "hero_id" на нулевую позицию
cols = ["hero_id"] + [col for col in df_final.columns if col != "hero_id"]
df_final = df_final[cols]

# Заменить hero_id на 76 для строк, где Hero = "Outworld Destroyer"
df_final.loc[df_final['Hero'] == "Outworld Destroyer", 'hero_id'] = 76

# Разделяем столбец Expert
df_final["Expert Win Rate"] = df_final["Expert"].str.extract(r"(\d+\.\d+)%").astype(float)
df_final["Expert Matches"] = df_final["Expert"].str.extract(r"\((\d+)\)").astype(int)

# Разделяем столбец WR 9500+ MMR
df_final["9500 Win Rate"] = df_final["WR 9500+ MMR"].str.extract(r"(\d+\.\d+)%").astype(float)
df_final["9500 Matches"] = df_final["WR 9500+ MMR"].str.extract(r" (\d+) ").astype(int)

# Удаляем старые столбцы (если нужно)
df_final.drop(columns=["Expert", "WR 9500+ MMR"], inplace=True)

# Результат
display(df_final)


,hero_id,Rank,Role,Hero,D2PT Rating,Matches,Win Rate,Contest Rate,Radiant,Dire,1st Phase,2nd Phase,Lastpick,Networth@10,localized_name,Expert Win Rate,Expert Matches,9500 Win Rate,9500 Matches
0,4,#1,pos 1,Bloodseeker,3449,3551,54.1,36.7,56.8,51.3,49.5,54.4,54.8,"{'main': 4305, 'range_min': 3625, 'range_max':...",Bloodseeker,59.5,788,53.5,1870
1,73,#2,pos 1,Alchemist,3389,4959,52.9,58.6,53.4,52.4,50.5,53.7,53.0,"{'main': 5571, 'range_min': 4649, 'range_max':...",Alchemist,57.7,889,54.4,2624
2,67,#3,pos 1,Spectre,3309,4542,52.5,63.7,53.7,51.2,50.3,52.8,53.8,"{'main': 3915, 'range_min': 3261, 'range_max':...",Spectre,57.1,609,54.2,2035
3,136,#4,pos 1,Marci,3098,278,57.9,19.5,54.1,60.9,65.0,50.9,51.9,"{'main': 4810, 'range_min': 3799, 'range_max':...",Marci,74.1,27,56.6,113
4,94,#5,pos 1,Medusa,3092,885,52.7,14.0,52.8,52.5,66.7,54.4,50.6,"{'main': 4495, 'range_min': 3817, 'range_max':...",Medusa,63.8,94,46.5,344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,108,#122,pos 5,Underlord,0,3,0.0,6.1,0.0,0.0,0.0,0.0,0.0,"{'main': 1818, 'range_min': 1599, 'range_max':...",Underlord,0.0,0,0.0,1
626,120,#123,pos 5,Pangolier,0,7,14.3,13.0,0.0,50.0,33.3,0.0,0.0,"{'main': 1604, 'range_min': 1604, 'range_max':...",Pangolier,0.0,0,16.7,6
627,129,#124,pos 5,Mars,0,15,53.3,17.4,50.0,57.1,62.5,60.0,0.0,"{'main': 2450, 'range_min': 1354, 'range_max':...",Mars,0.0,0,60.0,5
628,126,#125,pos 5,Void Spirit,0,10,10.0,7.7,25.0,0.0,16.7,0.0,0.0,"{'main': 2564, 'range_min': 1719, 'range_max':...",Void Spirit,0.0,0,0.0,2


In [49]:
# Приводим названия фасетов к единому регистру
df_full_facets["facet_upper"] = df_full_facets["facet"].str.upper()  # В верхний регистр
all_facets_df["name_upper"] = all_facets_df["name"].str.upper()  # В верхний регистр

# Выполняем объединение (merge) двух DataFrame по колонкам с названиями фасетов
df_final_facets = pd.merge(
    df_full_facets,
    all_facets_df[["facet_number", "id", "name_upper", "localized_name"]],  # Оставляем только нужные столбцы
    left_on=["facet_upper", "hero"],  # Соответствие по названию фасета и герою
    right_on=["name_upper", "localized_name"],  # Название фасета и героя
    how="left"  # Оставляем все строки из df_full_facets
)

# Удаляем временные колонки с верхним регистром (если они не нужны)
df_final_facets.drop(columns=["facet_upper", "name_upper"], inplace=True)

# Переименовываем столбец "id" в "hero_id"
df_final_facets.rename(columns={"id": "hero_id"}, inplace=True)

# Перемещаем "hero_id" на нулевую позицию
cols = ["hero_id"] + [col for col in df_final_facets.columns if col != "hero_id"]
df_final_facets = df_final_facets[cols]

# Итоговые столбцы
df_final_facets = df_final_facets[["hero_id", "hero", "facet", "Matches", "Win Rate", "Role", "facet_number"]]

# Результат
display(df_final_facets)


,hero_id,hero,facet,Matches,Win Rate,Role,facet_number
0,29,Tidehunter,KRAKEN SWELL,22,54.5,pos 1,1
1,37,Warlock,BLACK GRIMOIRE,1,100.0,pos 1,2
2,62,Bounty Hunter,THROUGH AND THROUGH,7,0.0,pos 1,1
3,71,Spirit Breaker,BULL RUSH,3,0.0,pos 1,1
4,86,Rubick,FRUGAL FILCH,2,0.0,pos 1,1
...,...,...,...,...,...,...,...
1315,56,Clinkz,SUPPRESSIVE FIRE,6,66.7,pos 5,1
1316,17,Storm Spirit,STATIC SLIDE,8,37.5,pos 5,2
1317,28,Slardar,LEG DAY,8,62.5,pos 5,1
1318,41,Faceless Void,TIME ZONE,10,40.0,pos 5,2


In [50]:
config = {
    "version": 3,
    "configs": [
        # Фасеты
        HeroConfigProcessor(df_final_facets, "Facet Matches>200", data_type="facet").build_config(matches_threshold=200, wr_threshold=51),
        HeroConfigProcessor(df_final_facets, "Facet Matches>50", data_type="facet").build_config(matches_threshold=50, wr_threshold=51),
        
        # Регулярные конфиги
        HeroConfigProcessor(df_final, "Win Rate", data_type="regular").build_config(matches_threshold=200, wr_threshold=51),
        HeroConfigProcessor(df_final, "D2PT", data_type="regular").build_config(matches_threshold=200, wr_threshold=0, sort_by="D2PT Rating"),
        
        # Эксперты
        HeroConfigProcessor(df_final, "Expert Win Rate", data_type="regular").build_config(
            expert_matches_threshold=50, expert_wr_threshold=51, sort_by="Expert Win Rate"
        ),
        
        # 9500 MMR
        HeroConfigProcessor(df_final, "9500 Win Rate", data_type="regular").build_config(
            mmr_9500_matches_threshold=100, mmr_9500_wr_threshold=51, sort_by="9500 Win Rate"
        ),
    ],
}


In [51]:
import threading


threading.Thread(target=lambda: driver.quit() if driver.service.process else None).start()

## Find Steam

In [52]:
import os
import winreg
import win32api

def read_reg(ep, p = r"", k = ''):
    try:
        key = winreg.OpenKeyEx(ep, p)
        value = winreg.QueryValueEx(key,k)
        if key:
            winreg.CloseKey(key)
        return value[0]
    except Exception as e:
        return None
    return None

Path1 = "{}\\Microsoft\\Windows\\Start Menu\\Programs\\Steam\\Steam.lnk".format(os.getenv('APPDATA'))
if os.path.exists(Path1):
    import sys
    import win32com.client 

    shell = win32com.client.Dispatch("WScript.Shell")
    shortcut = shell.CreateShortCut(Path1)
    Path1Res = shortcut.Targetpath
else:
    Path1Res = False
Path2 = str(read_reg(ep = winreg.HKEY_LOCAL_MACHINE, p = r"SOFTWARE\Wow6432Node\Valve\Steam", k = 'InstallPath'))+r"\steam.exe"
Path3 = str(read_reg(ep = winreg.HKEY_LOCAL_MACHINE, p = r"SOFTWARE\Valve\Steam", k = 'InstallPath'))+r"\steam.exe"
if not os.path.exists(Path2):
    Path2 = None
if not os.path.exists(Path3):
    Path3 = None
PossiblePaths = [r"X:\Steam\steam.exe", r"X:\Program Files\Steam\steam.exe", r"X:\Program Files (x86)\Steam\steam.exe"]
ValidHardPaths = []
for Drive in win32api.GetLogicalDriveStrings().split('\000')[:-1]:
    Drive = Drive.replace(':\\', '')
    for path in PossiblePaths:
        path = path.replace("X", Drive)
        if os.path.exists(path):
            ValidHardPaths.append(path)
if len(ValidHardPaths) == 0:
    ValidHardPaths = ["None"]
ValidHardPaths

['C:\\Program Files (x86)\\Steam\\steam.exe']

In [53]:
steam_dir = os.path.dirname(os.path.realpath(ValidHardPaths[0]))
steam_dir = os.path.join(steam_dir, "userdata")
steam_dir

'C:\\Program Files (x86)\\Steam\\userdata'

In [54]:
dirs_to_check = [os.path.join(steam_dir, id, "570", "remote", "cfg") for id in os.listdir(steam_dir)]
cfg_dirs = [dir for dir in dirs_to_check if os.path.isdir(dir)]
cfg_dirs

['C:\\Program Files (x86)\\Steam\\userdata\\106615943\\570\\remote\\cfg',
 'C:\\Program Files (x86)\\Steam\\userdata\\106675276\\570\\remote\\cfg',
 'C:\\Program Files (x86)\\Steam\\userdata\\114487406\\570\\remote\\cfg',
 'C:\\Program Files (x86)\\Steam\\userdata\\196954656\\570\\remote\\cfg',
 'C:\\Program Files (x86)\\Steam\\userdata\\302813473\\570\\remote\\cfg',
 'C:\\Program Files (x86)\\Steam\\userdata\\383613970\\570\\remote\\cfg']

In [55]:
from datetime import datetime

for dir in cfg_dirs:
    # Убедимся, что папка old_grid существует
    old_grid_dir = os.path.join(dir, "old_grid")
    os.makedirs(old_grid_dir, exist_ok=True)

    # Путь к текущему файлу конфигурации
    current_config_path = os.path.join(dir, "hero_grid_config.json")

    # Если текущий конфиг существует, переместим его в old_grid с отметкой времени
    if os.path.exists(current_config_path):
        timestamp = datetime.now().strftime("%Y%m%d")
        old_config_path = os.path.join(
            old_grid_dir, f"hero_grid_config_{timestamp}.json"
        )

        # Если файл с таким именем уже существует, удаляем его перед перемещением
        if os.path.exists(old_config_path):
            os.remove(old_config_path)

        os.rename(current_config_path, old_config_path)

    # Сохраняем новую версию конфигурации
    with open(current_config_path, "w") as f:
        json.dump(dict(config), f)

## Autorun

In [56]:
import getpass
import os
import sys
USER_NAME = getpass.getuser()

In [57]:
from win32com.client import Dispatch

bat_path = rf'C:\Users\{USER_NAME}\AppData\Roaming\Microsoft\Windows\Start Menu\Programs\Startup\{os.path.basename(sys.executable)}.lnk'
target = sys.executable

shell = Dispatch('WScript.Shell')
shortcut = shell.CreateShortCut(bat_path)
shortcut.Targetpath = target
shortcut.save()

In [58]:
print("Done!")

for remaining in range(2, 0, -1):
    time.sleep(1)

Done!
